In [0]:
from pyspark.sql.functions import count, countDistinct, col

# 讀取ga4_july_delta
ga4_df = spark.read.table("hive_metastore.default.ga4_july_delta")

# 篩選資料和計算pageview和users
ga4_result = ga4_df.agg(
    count("*").alias("pageview"),
    countDistinct("user_pseudo_id").alias("all_users"),
    countDistinct("user_id").alias("login_users")
)

ga4_result.show()


+--------+---------+-----------+
|pageview|all_users|login_users|
+--------+---------+-----------+
|  524006|    28798|       2841|
+--------+---------+-----------+



In [0]:
from pyspark.sql.functions import cast, count, sum

# 讀取order_july_delta
order_df = spark.read.table("hive_metastore.default.order_july_delta")

# 轉換sol_price為整數型態且過濾非空或非數值資料
order_df = order_df.withColumn("sol_price_int", col("sol_price").cast("int"))
order_df = order_df.filter(col("sol_price_int").isNotNull())

# 對整個表格進行加總
order_result = order_df.agg(
    count("*").alias("orders_num"),
    sum("sol_price_int").alias("sales")
)

order_result.show()


+----------+--------+
|orders_num|   sales|
+----------+--------+
|      2789|91556691|
+----------+--------+



In [0]:
from pyspark.sql.functions import col

# 讀取pagespeedinsight_delta表
df = spark.read.table("hive_metastore.default.pagespeedinsight_delta")

# 從結構型態中提取指定的欄位
df_extracted = df.select(
    col("lighthouseResult.finalUrl").alias("url"),
    col("loadingExperience.overall_category").alias("Overall_Category"),
    col("lighthouseResult.audits.`largest-contentful-paint`.displayValue").alias("Largest_Contentful_Paint"),
    col("loadingExperience.metrics.FIRST_INPUT_DELAY_MS.percentile").alias("First_Input_Delay"),
    col("lighthouseResult.audits.`cumulative-layout-shift`.displayValue").alias("Cumulative_Layout_Shift"),
    col("lighthouseResult.audits.`first-contentful-paint`.displayValue").alias("First_Contentful_Paint"),
    col("lighthouseResult.audits.interactive.displayValue").alias("Time_to_Interactive"),
    col("lighthouseResult.audits.`total-blocking-time`.displayValue").alias("Total_Blocking_Time"),
    col("lighthouseResult.audits.`speed-index`.displayValue").alias("Speed_Index")
)

# 顯示結果
df_extracted.show(truncate=False)


+--------------------------+----------------+------------------------+-----------------+-----------------------+----------------------+-------------------+-------------------+-----------+
|url                       |Overall_Category|Largest_Contentful_Paint|First_Input_Delay|Cumulative_Layout_Shift|First_Contentful_Paint|Time_to_Interactive|Total_Blocking_Time|Speed_Index|
+--------------------------+----------------+------------------------+-----------------+-----------------------+----------------------+-------------------+-------------------+-----------+
|https://www.tcloud.gov.tw/|AVERAGE         |4.6 s                   |3                |0.002                  |0.9 s                 |2.4 s              |190 ms             |1.2 s      |
+--------------------------+----------------+------------------------+-----------------+-----------------------+----------------------+-------------------+-------------------+-----------+



In [0]:
from pyspark.sql.functions import avg, sum, col, when

# 讀取searchconsole_july_delta
searchconsole_df = spark.read.table("hive_metastore.default.searchconsole_july_delta")

# 計算首頁平均排名
avg_position = searchconsole_df \
    .filter(col("page") == "https://www.tcloud.gov.tw/") \
    .agg(avg("position").alias("Homepage_Avg_Ranking"))

# 計算搜尋引擎曝光
total_impressions = searchconsole_df.agg(sum("impressions").alias("Search_Engine_Exposure"))

# 計算產品頁平均點擊率
product_page_click_rate = searchconsole_df \
    .filter(col("page").contains("/solution/")) \
    .agg((sum("clicks") / sum("impressions")).alias("Product_Page_Avg_ClickRate"))

# 合併結果為一個row
final_result = avg_position.crossJoin(total_impressions).crossJoin(product_page_click_rate) \
    .select("Homepage_Avg_Ranking", "Search_Engine_Exposure", "Product_Page_Avg_ClickRate")

final_result.show(truncate=False)


+--------------------+----------------------+--------------------------+
|Homepage_Avg_Ranking|Search_Engine_Exposure|Product_Page_Avg_ClickRate|
+--------------------+----------------------+--------------------------+
|57.273132335087034  |423481                |0.007263292466313936      |
+--------------------+----------------------+--------------------------+



In [0]:
from pyspark.sql.functions import lit

# 從df_extracted選擇指定的欄位
df_extracted_selected = df_extracted.select("Overall_Category", "Time_to_Interactive", "Speed_Index")

# 創建一個包含'2023-07'的month欄位
month_df = spark.createDataFrame([("2023-07",)], ["month"])

# 用crossJoin組合所有的結果
merged_df = month_df.crossJoin(ga4_result).crossJoin(order_result).crossJoin(df_extracted_selected).crossJoin(final_result)

# 顯示結果
merged_df.show(truncate=False)


+-------+--------+---------+-----------+----------+--------+----------------+-------------------+-----------+--------------------+----------------------+--------------------------+
|month  |pageview|all_users|login_users|orders_num|sales   |Overall_Category|Time_to_Interactive|Speed_Index|Homepage_Avg_Ranking|Search_Engine_Exposure|Product_Page_Avg_ClickRate|
+-------+--------+---------+-----------+----------+--------+----------------+-------------------+-----------+--------------------+----------------------+--------------------------+
|2023-07|524006  |28798    |2841       |2789      |91556691|AVERAGE         |2.4 s              |1.2 s      |57.273132335087034  |423481                |0.007263292466313936      |
+-------+--------+---------+-----------+----------+--------+----------------+-------------------+-----------+--------------------+----------------------+--------------------------+

